# 🔋 Analizador de Celdas Solares - Notebook Interactivo

**Análisis de Curvas I-V y P-V para Celdas Solares**

---

## 🎯 ¿Qué es esto?

Este notebook te permite analizar el rendimiento de celdas solares de forma fácil e interactiva. **No necesitas saber programación** - solo sigue las instrucciones paso a paso.

### 📊 Lo que hace este notebook:
- ✅ **Analiza tus datos** de voltaje y corriente de celdas solares
- ✅ **Calcula automáticamente** todos los parámetros importantes (Isc, Voc, Pmax, FF, eficiencia)
- ✅ **Genera gráficas profesionales** de las curvas I-V y P-V
- ✅ **Exporta resultados** en formato CSV para usar en Excel o similar

---

## 🚀 Cómo usar este notebook

### Para principiantes (sin experiencia en programación):

1. **📱 En Google Colab** (recomendado para principiantes):
   - Abre este archivo en [Google Colab](https://colab.research.google.com)
   - Haz clic en "Runtime" → "Run all" para ejecutar todo
   - Sigue las instrucciones que aparecen

2. **💻 En VS Code**:
   - Abre este archivo en VS Code
   - Haz clic en "Run All" en la parte superior
   - Sigue las instrucciones

3. **📓 En Jupyter Notebook**:
   - Abre este archivo en Jupyter
   - Haz clic en "Cell" → "Run All"
   - Sigue las instrucciones

### 📝 Preparación de datos

**Opción 1: Subir archivo CSV** (más fácil)
- Prepara tus datos en Excel o Google Sheets
- Guarda como archivo CSV con columnas: `Voltaje` y `Corriente`
- Sube el archivo cuando el notebook te lo pida

**Opción 2: Escribir datos directamente**
- El notebook te permitirá escribir tus datos manualmente
- Ideal para pocos puntos de medición

---

## 🆘 ¿Necesitas ayuda?

- **📖 Manual completo**: Consulta el archivo `README.md`
- **💬 Soporte**: Contacta a adrianarazo@outlook.com
- **🐛 Problemas**: Reporta issues en GitHub

---

## 🎓 Información académica

**Creado por**: Adriana Razo De León  
**Proyecto**: Análisis de Celdas Solares  
**Fecha**: 2025  
**Versión**: 2.0 (Notebook Interactivo)

---

## ▶️ ¡Empezar!

**Haz clic en la celda de abajo y presiona Shift+Enter para comenzar, o simplemente ejecuta todas las celdas.**

In [56]:
# 🔧 CONFIGURACIÓN INICIAL - Ejecuta esta celda primero
# Esta celda detecta el entorno y configura todo automáticamente

import sys
import os

# Detectar entorno
def detectar_entorno():
    """Detecta si estamos en Google Colab, VS Code, o Jupyter"""
    try:
        import google.colab
        return "Google Colab"
    except ImportError:
        pass
    
    if 'ipykernel' in sys.modules and 'VSCODE_PID' in os.environ:
        return "VS Code"
    elif 'ipykernel' in sys.modules:
        return "Jupyter Notebook"
    else:
        return "Python Script"

entorno = detectar_entorno()
print(f"🌟 Entorno detectado: {entorno}")

# Configurar según el entorno
if entorno == "Google Colab":
    print("📱 Configurando para Google Colab...")
    # En Colab, instalar dependencias automáticamente
    !pip install -q numpy matplotlib scipy pandas
    
elif entorno == "VS Code":
    print("💻 Configurando para VS Code...")
    # En VS Code, verificar dependencias
    try:
        import numpy, matplotlib, scipy, pandas
        print("✅ Todas las dependencias están instaladas")
    except ImportError as e:
        print(f"⚠️  Falta instalar: {e}")
        print("💡 Tip: Instala con: pip install numpy matplotlib scipy pandas")
        
elif entorno == "Jupyter Notebook":
    print("📓 Configurando para Jupyter Notebook...")
    # En Jupyter, verificar dependencias
    try:
        import numpy, matplotlib, scipy, pandas
        print("✅ Todas las dependencias están instaladas")
    except ImportError as e:
        print(f"⚠️  Falta instalar: {e}")
        print("💡 Tip: Instala con: pip install numpy matplotlib scipy pandas")

print("\n🎯 Configuración completada. ¡Continuemos!")
print("👇 Ejecuta la siguiente celda para comenzar el análisis")

🌟 Entorno detectado: Jupyter Notebook
📓 Configurando para Jupyter Notebook...
✅ Todas las dependencias están instaladas

🎯 Configuración completada. ¡Continuemos!
👇 Ejecuta la siguiente celda para comenzar el análisis


In [57]:
# 📚 IMPORTAR LIBRERÍAS
# Importamos todas las herramientas necesarias para el análisis

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import pandas as pd
from datetime import datetime
import csv
import io

# Configurar matplotlib para gráficas bonitas
plt.style.use('default')
plt.rcParams['font.size'] = 10
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['figure.facecolor'] = 'white'

# Función para cargar datos desde Google Colab
def cargar_desde_colab():
    """Función para cargar archivos en Google Colab"""
    from google.colab import files
    print("📁 Selecciona tu archivo CSV con datos de voltaje y corriente:")
    uploaded = files.upload()
    
    if uploaded:
        filename = list(uploaded.keys())[0]
        print(f"✅ Archivo '{filename}' cargado exitosamente")
        return filename
    else:
        print("❌ No se cargó ningún archivo")
        return None

# Función para cargar datos desde archivo local
def cargar_desde_local():
    """Función para cargar archivos locales"""
    print("📁 Archivos disponibles en el directorio:")
    archivos_csv = [f for f in os.listdir('.') if f.endswith('.csv')]
    
    if archivos_csv:
        print("Archivos CSV encontrados:")
        for i, archivo in enumerate(archivos_csv, 1):
            print(f"  {i}. {archivo}")
        
        while True:
            try:
                seleccion = input(f"Selecciona un archivo (1-{len(archivos_csv)}) o escribe el nombre: ")
                if seleccion.isdigit() and 1 <= int(seleccion) <= len(archivos_csv):
                    return archivos_csv[int(seleccion) - 1]
                elif seleccion in archivos_csv:
                    return seleccion
                elif os.path.exists(seleccion):
                    return seleccion
                else:
                    print("❌ Archivo no encontrado. Intenta de nuevo.")
            except KeyboardInterrupt:
                print("\\n❌ Operación cancelada")
                return None
    else:
        archivo = input("📝 Escribe el nombre del archivo CSV: ")
        if os.path.exists(archivo):
            return archivo
        else:
            print("❌ Archivo no encontrado")
            return None

print("✅ Librerías importadas correctamente")
print("🔧 Funciones auxiliares definidas")
print("\\n👇 Ejecuta la siguiente celda para continuar")

✅ Librerías importadas correctamente
🔧 Funciones auxiliares definidas
\n👇 Ejecuta la siguiente celda para continuar


In [65]:
# 🔬 FUNCIÓN PRINCIPAL DE ANÁLISIS
# Esta función hace todo el análisis automáticamente

def analizar_celda_solar(voltajes, corrientes, irradiancia=None, area=None, 
                        titulo="Análisis de Celda Solar", mostrar_graficas=True):
    """
    Analiza una celda solar y calcula todos los parámetros importantes
    
    Parámetros:
    - voltajes: Lista de valores de Volt (V)
    - corrientes: Lista de valores de corriente Ampere (A)
    - irradiancia: Irradiancia solar en W/m² (opcional)
    - area: Área de la celda en m² (opcional)
    - titulo: Título para las gráficas
    - mostrar_graficas: Si mostrar o no las gráficas
    
    Retorna: Diccionario con todos los resultados
    """
    
    # Convertir a arrays de NumPy
    V = np.array(voltajes, dtype=float)
    I = np.array(corrientes, dtype=float)
    
    print("=" * 60)
    print(f"🔋 {titulo}")
    print("=" * 60)
    print(f"📊 Analizando {len(V)} puntos de medición...")
    print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # 1. Calcular Isc (Corriente de cortocircuito)
    Isc = I[np.argmin(np.abs(V))]
    
    # 2. Calcular Voc (Voltaje de circuito abierto)
    try:
        f_iv = interpolate.interp1d(I, V, kind='linear', fill_value='extrapolate')
        Voc = float(f_iv(0.0))
    except:
        Voc = V[I > 0][-1] if len(V[I > 0]) > 0 else V[-1]
    
    # 3. Calcular potencia y punto de máxima potencia
    P = V * I
    idx_max = np.argmax(P)
    Vmp, Imp, Pmax = V[idx_max], I[idx_max], P[idx_max]
    
    # 4. Calcular Factor de Llenado (Fill Factor)
    FF = Pmax / (Isc * Voc) if (Isc > 0 and Voc > 0) else 0
    
    # 5. Calcular eficiencia (si hay datos)
    eficiencia = None
    if irradiancia and area and irradiancia > 0 and area > 0:
        Pin = irradiancia * area
        eficiencia = Pmax / Pin if Pin > 0 else 0
    
    # Mostrar resultados
    print("📋 RESULTADOS DEL ANÁLISIS:")
    print("-" * 50)
    print(f"🔌 Corriente de cortocircuito (Isc): {Isc:.4f} A")
    print(f"⚡ Voltaje de circuito abierto (Voc): {Voc:.4f} V")
    print(f"🔋 Punto de máxima potencia:")
    print(f"   • Corriente (Imp): {Imp:.4f} A")
    print(f"   • Voltaje (Vmp): {Vmp:.4f} V")
    print(f"   • Potencia (Pmax): {Pmax:.4f} W")
    print(f"📊 Factor de llenado (FF): {FF*100:.2f}%")
    
    if eficiencia is not None:
        print(f"🌟 Eficiencia (η): {eficiencia*100:.2f}%")
    else:
        print("ℹ️  Eficiencia no calculada (faltan datos de irradiancia o área)")
    
    # Crear diccionario con resultados
    resultados = {
        'Isc': Isc,
        'Voc': Voc,
        'Imp': Imp,
        'Vmp': Vmp,
        'Pmax': Pmax,
        'FF': FF,
        'Eficiencia': eficiencia,
        'Voltajes': V,
        'Corrientes': I,
        'Potencias': P,
        'Titulo': titulo
    }
    
    # Generar gráficas si se solicita
    if mostrar_graficas:
        generar_graficas(resultados)
    return resultados

def generar_graficas(resultados):
    """Genera las gráficas I-V y P-V"""
    V = resultados['Voltajes']
    I = resultados['Corrientes']
    P = resultados['Potencias']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle(resultados['Titulo'], fontsize=16, fontweight='bold')

    # Gráfica I-V
    ax1.plot(V, I, '-o', linewidth=2, markersize=6, label='Curva I-V', color='#2E86AB')
    ax1.plot(resultados['Vmp'], resultados['Imp'], 'ko', markersize=10, 
             label=f'Pmax = {resultados["Pmax"]:.3f} W')
    ax1.plot([0, resultados['Voc']], [resultados['Isc'], 0], 'r--', 
             linewidth=2, alpha=0.7, label='Rectángulo ideal')

    # Líneas del punto de máxima potencia
    ax1.plot([resultados['Vmp'], resultados['Vmp']], [0, resultados['Imp']], 
             'k--', linewidth=1, alpha=0.5)
    ax1.plot([0, resultados['Vmp']], [resultados['Imp'], resultados['Imp']], 
             'k--', linewidth=1, alpha=0.5)

    ax1.set_title("Curva Corriente-Voltaje (I-V)", fontweight='bold')
    ax1.set_xlabel("Voltaje (V)")
    ax1.set_ylabel("Corriente (A)")
    ax1.grid(True, alpha=0.3)
    ax1.legend()

    # Gráfica P-V
    ax2.plot(V, P, '-o', linewidth=2, markersize=6, color='#A23B72', label='Curva P-V')
    ax2.plot(resultados['Vmp'], resultados['Pmax'], 'ks', markersize=10, 
             label=f'Pmax = {resultados["Pmax"]:.3f} W')

    ax2.set_title("Curva Potencia-Voltaje (P-V)", fontweight='bold')
    ax2.set_xlabel("Voltaje (V)")
    ax2.set_ylabel("Potencia (W)")
    ax2.grid(True, alpha=0.3)
    ax2.legend()

    plt.tight_layout()

    # Mostrar la figura solo si está en un entorno interactivo
    import sys
    if hasattr(sys, 'ps1') or 'ipykernel' in sys.modules:
        plt.show()
    else:
        print("⚠️  plt.show() omitido (entorno no interactivo)")

print("✅ Función de análisis definida")
print("🔬 Lista para analizar datos de celdas solares")
print("\n👇 Ejecuta la siguiente celda para comenzar a cargar datos")

✅ Función de análisis definida
🔬 Lista para analizar datos de celdas solares

👇 Ejecuta la siguiente celda para comenzar a cargar datos


## 📂 PASO 1: Cargar tus datos

Tienes **3 opciones** para proporcionar tus datos:

### 📝 Opción A: Usar datos de ejemplo (para probar)
Si quieres probar el notebook primero, puedes usar datos de ejemplo incluidos.

### 📁 Opción B: Cargar archivo CSV (recomendado)
Si tienes tus datos en Excel o Google Sheets:
1. Guarda como archivo CSV
2. Asegúrate de que tenga columnas: `Voltaje` y `Corriente`
3. Carga el archivo en la celda siguiente

### ✏️ Opción C: Escribir datos manualmente
Si tienes pocos datos, puedes escribirlos directamente en el notebook.

---

### 📋 Formato de datos CSV esperado:

```csv
Voltaje,Corriente
0.0,0.500
0.1,0.480
0.2,0.450
0.3,0.400
0.4,0.300
0.5,0.000
```

**Importante:**
- Los datos pueden estar en cualquier orden
- Puede usar comas o puntos para decimales
- Puede tener encabezados o no
- Separadores: coma (,), punto y coma (;), o tabulación

---

**👇 Ejecuta la siguiente celda para elegir cómo cargar tus datos**

In [59]:
# 🎯 SELECCIONAR MÉTODO DE CARGA DE DATOS
# Ejecuta esta celda y sigue las instrucciones

def cargar_datos_csv(archivo):
    """Carga datos desde un archivo CSV"""
    try:
        # Intentar detectar separador
        with open(archivo, 'r', encoding='utf-8') as f:
            sample = f.read(1024)
            if ';' in sample:
                sep = ';'
            elif '\t' in sample:
                sep = '\t'
            else:
                sep = ','
        
        # Cargar datos
        df = pd.read_csv(archivo, sep=sep)
        
        # Buscar columnas de voltaje y corriente
        voltage_cols = ['V', 'Voltage', 'Voltaje', 'voltage', 'v', 'voltaje']
        current_cols = ['I', 'Current', 'Corriente', 'current', 'i', 'corriente']
        
        voltage_col = None
        current_col = None
        
        for col in df.columns:
            col_clean = col.strip()
            if any(vc in col_clean for vc in voltage_cols) or 'volt' in col_clean.lower():
                voltage_col = col
            elif any(cc in col_clean for cc in current_cols) or 'curr' in col_clean.lower() or 'amp' in col_clean.lower():
                current_col = col
        
        # Si no se encuentran, usar las primeras dos columnas
        if voltage_col is None or current_col is None:
            if len(df.columns) >= 2:
                voltage_col = df.columns[0]
                current_col = df.columns[1]
                print(f"⚠️  Usando columnas: '{voltage_col}' como voltaje, '{current_col}' como corriente")
        
        voltajes = df[voltage_col].dropna().tolist()
        corrientes = df[current_col].dropna().tolist()
        
        print(f"✅ Datos cargados: {len(voltajes)} puntos")
        print(f"📊 Rango voltaje: {min(voltajes):.3f} - {max(voltajes):.3f} V")
        print(f"📊 Rango corriente: {min(corrientes):.6f} - {max(corrientes):.6f} A")
        
        return voltajes, corrientes
        
    except Exception as e:
        print(f"❌ Error al cargar archivo: {e}")
        return None, None

# Variables globales para almacenar datos
voltajes_datos = None
corrientes_datos = None

print("🎯 Selecciona el método para cargar tus datos:")
print("1️⃣  Usar datos de ejemplo (para probar)")
print("2️⃣  Cargar archivo CSV")
print("3️⃣  Escribir datos manualmente")
print()

# Detectar entorno para método de carga
if entorno == "Google Colab":
    print("📱 Estás en Google Colab - podrás subir archivos fácilmente")
elif entorno == "VS Code":
    print("💻 Estás en VS Code - usa archivos locales")
else:
    print("📓 Estás en Jupyter Notebook - usa archivos locales")

print("\\n👇 Ejecuta la siguiente celda para elegir una opción")

🎯 Selecciona el método para cargar tus datos:
1️⃣  Usar datos de ejemplo (para probar)
2️⃣  Cargar archivo CSV
3️⃣  Escribir datos manualmente

📓 Estás en Jupyter Notebook - usa archivos locales
\n👇 Ejecuta la siguiente celda para elegir una opción


In [ ]:
# 1️⃣ OPCIÓN A: USAR DATOS DE EJEMPLO
# Ejecuta esta celda solo si quieres usar datos de ejemplo para probar

def usar_datos_ejemplo():
    """Carga datos de ejemplo de una celda solar típica"""
    global voltajes_datos, corrientes_datos
    
    # Datos de ejemplo (celda solar típica)
    voltajes_datos = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
    corrientes_datos = [0.52, 0.51, 0.50, 0.49, 0.48, 0.46, 0.43, 0.39, 0.33, 0.25, 0.15, 0.0]
    
    print("✅ Datos de ejemplo cargados exitosamente")
    print(f"📊 {len(voltajes_datos)} puntos de medición")
    print(f"📈 Rango voltaje: {min(voltajes_datos):.3f} - {max(voltajes_datos):.3f} V")
    print(f"📈 Rango corriente: {min(corrientes_datos):.3f} - {max(corrientes_datos):.3f} A")
    print()
    print("🎯 Datos listos para análisis!")
    print("👇 Salta a la sección 'PASO 2' para continuar")

# Descomenta la siguiente línea para usar datos de ejemplo:
# usar_datos_ejemplo()

In [4]:
# 🎯 INICIO RÁPIDO - ¡Usa esto para empezar inmediatamente!
# Ejecuta esta celda para usar datos de ejemplo automáticamente

def inicio_rapido():
    """Función para comenzar inmediatamente con datos de ejemplo"""
    global voltajes_datos, corrientes_datos, irradiancia, area_celda, titulo_analisis
    
    print("🚀 INICIO RÁPIDO - Cargando datos de ejemplo...")
    
    # Cargar datos de ejemplo automáticamente
    voltajes_datos = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
    corrientes_datos = [0.52, 0.51, 0.50, 0.49, 0.48, 0.46, 0.43, 0.39, 0.33, 0.25, 0.15, 0.0]
    
    # Configurar parámetros automáticamente
    irradiancia = 1000  # W/m²
    area_celda = 0.0001  # m² (1 cm²)
    titulo_analisis = "Análisis de Celda Solar - Ejemplo"
    
    print("✅ Datos de ejemplo cargados")
    print(f"📊 {len(voltajes_datos)} puntos de medición")
    print(f"🌞 Irradiancia: {irradiancia} W/m²")
    print(f"📏 Área: {area_celda} m² (1 cm²)")
    print()
    print("🎯 ¡Todo listo! Ahora ejecuta el análisis completo abajo ↓")
    print("👇 Busca la celda '🚀 ANÁLISIS COMPLETO' y ejecútala")

# 🚀 EJECUTAR INICIO RÁPIDO
# Descomenta la siguiente línea para usar el inicio rápido:
inicio_rapido()

🚀 INICIO RÁPIDO - Cargando datos de ejemplo...
✅ Datos de ejemplo cargados
📊 12 puntos de medición
🌞 Irradiancia: 1000 W/m²
📏 Área: 0.0001 m² (1 cm²)

🎯 ¡Todo listo! Ahora ejecuta el análisis completo abajo ↓
👇 Busca la celda '🚀 ANÁLISIS COMPLETO' y ejecútala


In [66]:
# 2️⃣ OPCIÓN B: CARGAR ARCHIVO CSV/TSV
# Ejecuta esta celda para cargar cualquier archivo con datos de voltaje y corriente

def cargar_csv(ruta_archivo=None):
    """Carga datos desde archivo CSV o TSV según el entorno"""
    global voltajes_datos, corrientes_datos
    
    # Si estamos en Google Colab, permitir carga desde el navegador
    if entorno == "Google Colab":
        try:
            from google.colab import files
            print("📤 Subir archivo desde tu computadora:")
            print("📋 El archivo debe tener dos columnas: Voltaje y Corriente")
            print("   (Puede usar separadores de coma, punto y coma o tabulación)")
            
            uploaded = files.upload()
            if uploaded:
                archivo_subido = list(uploaded.keys())[0]
                print(f"✅ Archivo '{archivo_subido}' subido exitosamente")
                procesar_archivo(archivo_subido)
                return
        except Exception as e:
            print(f"❌ Error al subir archivo: {e}")
    
    # Primero, buscar el archivo específico si se proporcionó
    archivo_encontrado = None
    
    if ruta_archivo:
        # Lista de posibles ubicaciones donde buscar
        ubicaciones = [
            ruta_archivo,  # Ruta exacta
            os.path.join("datos csv", ruta_archivo),  # En subdirectorio "datos csv"
            os.path.join("datos_csv", ruta_archivo),  # Sin espacios
            os.path.join("..", "datos csv", ruta_archivo)  # Un nivel arriba
        ]
        
        # Buscar en todas las posibles ubicaciones
        for ubicacion in ubicaciones:
            if os.path.exists(ubicacion):
                archivo_encontrado = ubicacion
                break
    
    if archivo_encontrado:
        print(f"📂 Archivo encontrado: {archivo_encontrado}")
        procesar_archivo(archivo_encontrado)
        return
    
    # Si no se encontró el archivo específico, mostrar opciones
    print("📂 Selecciona un archivo para analizar:")
    print("1️⃣ Usa un archivo existente")
    print("2️⃣ Sube un nuevo archivo")
    
    opcion = input("Ingresa tu opción (1 o 2): ")
    
    if opcion == "1":
        # Buscar archivos existentes
        buscar_archivos_existentes()
    elif opcion == "2":
        # Instrucciones para subir un archivo nuevo
        if entorno == "Google Colab":
            print("👆 Ejecuta esta celda nuevamente para ver la opción de subir archivos")
        else:
            print("📝 Para subir un archivo nuevo:")
            print("1. Coloca tu archivo en la carpeta 'datos csv'")
            print("2. Ejecuta esta celda nuevamente")
            print("3. Selecciona la opción 1 para usar un archivo existente")
    else:
        print("❌ Opción no válida")
        print("💡 Intenta ejecutar usando datos de ejemplo con 'inicio_rapido()'")

def procesar_archivo(ruta_archivo):
    """Procesa un archivo y extrae los datos de voltaje y corriente"""
    global voltajes_datos, corrientes_datos
    
    try:
        # Detectar formato del archivo
        print(f"🔍 Analizando formato del archivo: {ruta_archivo}")
        
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            primeras_lineas = [f.readline() for _ in range(5)]
        
        # Detectar separador
        separador = '\t'  # Por defecto tabulación como en 1368h.csv
        for linea in primeras_lineas:
            if '\t' in linea:
                separador = '\t'
                print("✅ Formato detectado: Archivo separado por tabulaciones (TSV)")
                break
            elif ';' in linea:
                separador = ';'
                print("✅ Formato detectado: CSV con punto y coma")
                break
            elif ',' in linea:
                separador = ','
                print("✅ Formato detectado: CSV con comas")
                break
        
        # Leer archivo y extraer datos
        voltajes = []
        corrientes = []
        
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            lineas = f.readlines()
            
            # Verificar si hay líneas de comentario
            tiene_comentarios = any(linea.strip().startswith('//') for linea in lineas[:5])
            if tiene_comentarios:
                print("✅ El archivo contiene líneas de comentario")
            
            # Verificar si tiene encabezados
            tiene_encabezados = False
            for i, linea in enumerate(lineas):
                if not linea.strip().startswith('//'):
                    limpia = linea.strip().replace('"', '')
                    if 'voltaje' in limpia.lower() or 'corriente' in limpia.lower():
                        tiene_encabezados = True
                        print("✅ El archivo contiene encabezados")
                    break
            
            # Procesar cada línea
            for i, linea in enumerate(lineas):
                # Saltar comentarios
                if linea.strip().startswith('//'):
                    continue
                
                # Limpiar línea (quitar comillas y espacios)
                linea_limpia = linea.strip().replace('"', '')
                
                # Saltar encabezados
                if i == 0 and tiene_encabezados and ('voltaje' in linea_limpia.lower() or 'corriente' in linea_limpia.lower()):
                    continue
                
                # Dividir según el separador
                partes = linea_limpia.split(separador)
                
                # Extraer datos si hay al menos dos columnas
                if len(partes) >= 2:
                    try:
                        v = float(partes[0])
                        c = float(partes[1])
                        voltajes.append(v)
                        corrientes.append(c)
                    except ValueError:
                        # Probablemente encabezado o texto, ignorar
                        pass
        
        # Verificar que se hayan encontrado datos
        if voltajes and corrientes:
            # Guardar en variables globales
            voltajes_datos = voltajes
            corrientes_datos = corrientes
            
            print(f"✅ Datos cargados: {len(voltajes)} puntos")
            print(f"📊 Rango voltaje: {min(voltajes):.3f} - {max(voltajes):.3f} V")
            print(f"📊 Rango corriente: {min(corrientes):.6f} - {max(corrientes):.6f} A")
            
            # Mostrar vista previa de los datos
            print("\n📋 Vista previa de los datos (primeros 5 puntos):")
            print("   Voltaje   |   Corriente")
            print("  -----------|-------------")
            for v, c in zip(voltajes[:5], corrientes[:5]):
                print(f"   {v:8.3f}  |   {c:8.3f}")
            if len(voltajes) > 5:
                print(f"   ... y {len(voltajes)-5} puntos más")
            
            print("\n🎯 Archivo cargado exitosamente!")
            print("👇 Continúa con el 'PASO 2'")
        else:
            print("❌ No se encontraron datos válidos en el archivo")
            print("💡 Asegúrate de que el archivo tenga el formato correcto")
            
    except Exception as e:
        print(f"❌ Error al procesar archivo: {e}")
        import traceback
        traceback.print_exc()

def buscar_archivos_existentes():
    """Busca archivos CSV/TSV en el sistema de archivos"""
    # Buscar en varias ubicaciones
    archivos_encontrados = []
    
    # 1. Buscar en "datos csv"
    datos_dir = "datos csv"
    if os.path.exists(datos_dir) and os.path.isdir(datos_dir):
        for file in os.listdir(datos_dir):
            if file.endswith(('.csv', '.CSV', '.tsv', '.TSV')):
                ruta = os.path.join(datos_dir, file)
                archivos_encontrados.append(ruta)
    
    # 2. Buscar en directorio actual
    for file in os.listdir('.'):
        if file.endswith(('.csv', '.CSV', '.tsv', '.TSV')):
            archivos_encontrados.append(file)
    
    # Mostrar archivos encontrados
    if archivos_encontrados:
        print("\n📁 Archivos encontrados:")
        for i, archivo in enumerate(archivos_encontrados, 1):
            print(f"  {i}. {archivo}")
        
        # Permitir selección
        try:
            seleccion = input(f"Selecciona un archivo (1-{len(archivos_encontrados)}): ")
            if seleccion.isdigit() and 1 <= int(seleccion) <= len(archivos_encontrados):
                archivo_seleccionado = archivos_encontrados[int(seleccion) - 1]
                print(f"\n📂 Usando archivo: {archivo_seleccionado}")
                procesar_archivo(archivo_seleccionado)
            else:
                print("❌ Selección no válida")
                print("💡 Usando el primer archivo encontrado")
                procesar_archivo(archivos_encontrados[0])
        except Exception as e:
            print(f"❌ Error al seleccionar: {e}")
            if archivos_encontrados:
                print(f"💡 Usando el primer archivo: {archivos_encontrados[0]}")
                procesar_archivo(archivos_encontrados[0])
    else:
        print("❌ No se encontraron archivos CSV/TSV")
        print("💡 Coloca tu archivo en la carpeta 'datos csv' o en el directorio actual")
        print("💡 Intenta ejecutar usando datos de ejemplo con 'inicio_rapido()'")

# Ejecutar el cargador de archivos - sin archivos específicos predeterminados
try:
    print("🔍 Buscando archivos CSV/TSV disponibles...")
    # Llamar a cargar_csv sin parámetros para permitir selección de cualquier archivo
    cargar_csv()
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    print("💡 Intenta ejecutar usando datos de ejemplo con 'inicio_rapido()'")
    print("💡 O coloca tu archivo CSV/TSV en la carpeta 'datos csv'")

🔍 Buscando archivos CSV/TSV disponibles...
📂 Selecciona un archivo para analizar:
1️⃣ Usa un archivo existente
2️⃣ Sube un nuevo archivo

📁 Archivos encontrados:
  1. datos csv/1368h.csv
  2. datos csv/2400h.csv

📂 Usando archivo: datos csv/1368h.csv
🔍 Analizando formato del archivo: datos csv/1368h.csv
✅ Formato detectado: Archivo separado por tabulaciones (TSV)
✅ El archivo contiene encabezados
✅ Datos cargados: 151 puntos
📊 Rango voltaje: -0.500 - 1.000 V
📊 Rango corriente: -17.626220 - 199.246960 A

📋 Vista previa de los datos (primeros 5 puntos):
   Voltaje   |   Corriente
  -----------|-------------
     -0.500  |    -17.626
     -0.490  |    -17.607
     -0.480  |    -17.624
     -0.470  |    -17.603
     -0.460  |    -17.530
   ... y 146 puntos más

🎯 Archivo cargado exitosamente!
👇 Continúa con el 'PASO 2'

📁 Archivos encontrados:
  1. datos csv/1368h.csv
  2. datos csv/2400h.csv

📂 Usando archivo: datos csv/1368h.csv
🔍 Analizando formato del archivo: datos csv/1368h.csv
✅ For

In [ ]:
# 3️⃣ OPCIÓN C: ESCRIBIR DATOS MANUALMENTE
# Ejecuta esta celda solo si quieres escribir tus datos directamente

def escribir_datos_manualmente():
    """Permite escribir datos manualmente"""
    global voltajes_datos, corrientes_datos
    
    print("✏️ Escribiendo datos manualmente...")
    print("📝 Escribe tus datos de voltaje y corriente")
    print("💡 Tip: Separa los valores con comas")
    print()
    
    # Ejemplo de datos para mostrar el formato
    print("📋 Ejemplo de formato:")
    print("Voltajes: 0.0, 0.1, 0.2, 0.3, 0.4, 0.5")
    print("Corrientes: 0.50, 0.48, 0.45, 0.40, 0.30, 0.00")
    print()
    
    # Datos manuales - CAMBIA ESTOS VALORES POR TUS DATOS REALES
    voltajes_texto = "0.0, 0.1, 0.2, 0.3, 0.4, 0.5"
    corrientes_texto = "0.50, 0.48, 0.45, 0.40, 0.30, 0.00"
    
    print("📝 Modifica los siguientes valores en el código:")
    print(f"voltajes_texto = \"{voltajes_texto}\"")
    print(f"corrientes_texto = \"{corrientes_texto}\"")
    print()
    
    try:
        # Procesar datos
        voltajes_datos = [float(x.strip()) for x in voltajes_texto.split(',')]
        corrientes_datos = [float(x.strip()) for x in corrientes_texto.split(',')]
        
        # Validar que tengan el mismo número de puntos
        if len(voltajes_datos) != len(corrientes_datos):
            print("❌ Error: Número diferente de voltajes y corrientes")
            return
        
        if len(voltajes_datos) < 3:
            print("❌ Error: Se necesitan al menos 3 puntos de medición")
            return
        
        print("✅ Datos escritos exitosamente")
        print(f"📊 {len(voltajes_datos)} puntos de medición")
        print(f"📈 Rango voltaje: {min(voltajes_datos):.3f} - {max(voltajes_datos):.3f} V")
        print(f"📈 Rango corriente: {min(corrientes_datos):.3f} - {max(corrientes_datos):.3f} A")
        print()
        print("🎯 Datos listos para análisis!")
        print("👇 Continúa con el 'PASO 2'")
        
    except Exception as e:
        print(f"❌ Error al procesar datos: {e}")
        print("💡 Revisa el formato de los datos")

# Descomenta la siguiente línea para escribir datos manualmente:
# escribir_datos_manualmente()

## ⚙️ PASO 2: Configurar parámetros de análisis

Una vez que hayas cargado tus datos, puedes configurar parámetros adicionales para un análisis más completo.

### 📊 Parámetros opcionales:

- **🌞 Irradiancia**: Potencia solar incidente (W/m²)
  - Valor estándar: 1000 W/m² (condiciones STC)
  - Necesario para calcular eficiencia

- **📏 Área de la celda**: Área activa de la celda solar (m²)
  - Ejemplo: 1 cm² = 0.0001 m²
  - Necesario para calcular eficiencia

- **🏷️ Título**: Nombre para identificar tu análisis

### 🔍 ¿Qué calculará el notebook?

✅ **Siempre calcula:**
- Corriente de cortocircuito (Isc)
- Voltaje de circuito abierto (Voc)
- Punto de máxima potencia (Imp, Vmp, Pmax)
- Factor de llenado (FF)
- Gráficas I-V y P-V

✅ **Si proporcionas irradiancia y área:**
- Eficiencia de conversión (η)
- Densidad de potencia

---

**👇 Ejecuta la siguiente celda para configurar los parámetros**

In [67]:
# ⚙️ CONFIGURAR PARÁMETROS DE ANÁLISIS
# Modifica estos valores según tu experimento

# Estos parámetros son opcionales y solo se usan para eficiencia si los proporcionas
if 'irradiancia' not in globals():
    irradiancia = None  # W/m² (opcional, solo para eficiencia)

if 'area_celda' not in globals():
    area_celda = None  # m² (opcional, solo para eficiencia)

# 🏷️ Título para identificar tu análisis
if 'titulo_analisis' not in globals():
    titulo_analisis = "Análisis de Celda Solar"

# 📊 Configuraciones adicionales
mostrar_graficas = True  # Mostrar gráficas I-V y P-V

print("⚙️ Configuración de parámetros:")
print(f"🏷️ Título: {titulo_analisis}")
print(f"📊 Mostrar gráficas: {mostrar_graficas}")
if irradiancia:
    print(f"🌞 Irradiancia: {irradiancia} W/m² (opcional)")
if area_celda:
    print(f"📏 Área de celda: {area_celda} m² (opcional)")
print()

# Validar que se hayan cargado datos
if 'voltajes_datos' in globals() and voltajes_datos is not None:
    print("✅ Datos cargados correctamente")
    print(f"📊 Puntos de medición: {len(voltajes_datos)}")
    print()
    print("🎯 Configuración completa!")
    print("👇 Ejecuta la siguiente celda para realizar el análisis")
else:
    print("❌ No se han cargado datos aún")
    print("🔄 Regresa al PASO 1 para cargar tus datos primero")
    print("💡 Tip: Usa 'inicio_rapido()' para empezar inmediatamente")

⚙️ Configuración de parámetros:
🏷️ Título: Análisis de Celda Solar
📊 Mostrar gráficas: True
🌞 Irradiancia: 1000 W/m² (opcional)
📏 Área de celda: 0.0001 m² (opcional)

✅ Datos cargados correctamente
📊 Puntos de medición: 151

🎯 Configuración completa!
👇 Ejecuta la siguiente celda para realizar el análisis


## 🚀 PASO 3: Ejecutar el análisis

¡Ahora viene la parte emocionante! El notebook analizará tus datos y te mostrará:

### 📊 Resultados que obtendrás:

1. **📈 Gráficas profesionales**:
   - Curva I-V (Corriente vs Voltaje)
   - Curva P-V (Potencia vs Voltaje)
   - Puntos característicos marcados

2. **🔢 Parámetros calculados**:
   - **Isc**: Corriente de cortocircuito
   - **Voc**: Voltaje de circuito abierto
   - **Imp, Vmp**: Corriente y voltaje de máxima potencia
   - **Pmax**: Potencia máxima
   - **FF**: Factor de llenado (Fill Factor)
   - **η**: Eficiencia (si configuraste irradiancia y área)

3. **💾 Archivos generados**:
   - Gráficas en formato PNG de alta calidad
   - Resultados en formato CSV para Excel

### 🎯 ¿Qué significan los resultados?

- **Isc**: Máxima corriente que puede generar la celda
- **Voc**: Máximo voltaje que puede generar la celda
- **Pmax**: Máxima potencia que puede generar la celda
- **FF**: Qué tan "cuadrada" es la curva I-V (ideal = 100%)
- **η**: Qué porcentaje de luz solar se convierte en electricidad

---

**👇 Ejecuta la siguiente celda para realizar el análisis completo**

In [68]:
# 🚀 ANÁLISIS COMPLETO DE LA CELDA SOLAR
# Esta celda ejecuta todo el análisis y muestra los resultados

def calcular_parametros_iv(V, I):
    """
    Calcula Jsc, Voc, Pmax, Vmp, Imp, FF y PCE a partir de una curva I-V.
    V: voltajes (array)
    I: corrientes (array)
    Retorna: diccionario con los parámetros.
    """
    # Usar corriente absoluta para cálculos físicos
    I_inver = -(I)

    # 1. Jsc (Densidad de corriente de cortocircuito): I(V=0)
    try:
        f_v = interpolate.interp1d(V, I, kind='linear', fill_value='extrapolate')
        Jsc = float(f_v(0.0))
    except Exception:
        Jsc = I[np.argmin(np.abs(V))]

    # 2. Voc (Voltaje de circuito abierto): V(I=0)
    try:
        f_i = interpolate.interp1d(I, V, kind='linear', fill_value='extrapolate')
        Voc = float(f_i(0.0))
    except Exception:
        Voc = V[np.argmax(V)]

    # 3. Pmax, Vmp, Imp (usar corriente absoluta para potencia útil)
    P = V * I_inver
    idx_max = np.argmax(P)
    Vmp = V[idx_max]
    Imp = I_inver[idx_max]
    Pmax = Vmp * Imp

    # 4. FF (Factor de llenado, %)
    if Jsc != 0 and Voc != 0:
        FF = (abs(Pmax)) / (abs(Jsc) * abs(Voc)) * 100
    else:
        FF = 0

    # 5. PCE relativa (sin área ni irradiancia)
    PCE = abs(Jsc) * abs(Voc) * FF / 100

    return {
        'Jsc': abs(Jsc),  # Siempre positivo
        'Voc': abs(Voc),  # Siempre positivo
        'Imp': abs(Imp),
        'Vmp': abs(Vmp),
        'Pmax': abs(Pmax),
        'FF': FF,
        'PCE': PCE,
        'Potencias': P
    }

def ejecutar_analisis_completo():
    if 'voltajes_datos' not in globals() or voltajes_datos is None:
        print("❌ Error: No se han cargado datos")
        print("🔄 Regresa al PASO 1 para cargar tus datos primero")
        return None

    print("🚀 Iniciando análisis completo...")
    print("=" * 70)

    V = np.array(voltajes_datos, dtype=float)
    I = np.array(corrientes_datos, dtype=float)

    # Calcular parámetros principales
    params = calcular_parametros_iv(V, I)
    Jsc = params['Jsc']
    Voc = params['Voc']
    Imp = params['Imp']
    Vmp = params['Vmp']
    Pmax = params['Pmax']
    FF = params['FF']
    PCE = params['PCE']
    P = params['Potencias']

    # Si el usuario proporciona irradiancia y área, calcula eficiencia absoluta
    eficiencia = None
    if 'irradiancia' in globals() and 'area_celda' in globals() and irradiancia and area_celda and irradiancia > 0 and area_celda > 0:
        Pin = irradiancia * area_celda
        eficiencia = Pmax / Pin if Pin > 0 else None
    else:
        eficiencia = PCE  # PCE relativa (%)

    print("\n📊 RESULTADOS DEL ANÁLISIS")
    print("=" * 70)
    print(f"🔋 Celda analizada: {titulo_analisis if 'titulo_analisis' in globals() else ''}")
    print(f"📅 Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Puntos de medición: {len(V)}")
    print()
    print("📈 PARÁMETROS PRINCIPALES:")
    print(f"  • Jsc (Densidad de corriente cortocircuito): {Jsc:.4f} mA/cm²")
    print(f"  • Voc (Voltaje circuito abierto): {Voc:.4f} V")
    print(f"  • Imp (Corriente máxima potencia): {Imp:.4f} mA/cm²")
    print(f"  • Vmp (Voltaje máxima potencia): {Vmp:.4f} V")
    print(f"  • Pmax (Potencia máxima): {Pmax:.4f} mW/cm²")
    print(f"  • FF (Factor de llenado): {FF:.2f}%")
    print(f"  • η (Eficiencia/PCE relativa): {eficiencia:.2f}%")

    # Mostrar gráficas usando la función existente
    resultados = {
        'Isc': Jsc,
        'Voc': Voc,
        'Imp': Imp,
        'Vmp': Vmp,
        'Pmax': Pmax,
        'FF': FF,
        'Eficiencia': eficiencia,
        'Voltajes': V,
        'Corrientes': I,
        'Potencias': P,
        'Titulo': titulo_analisis if 'titulo_analisis' in globals() else 'Análisis de Celda Solar'
    }
    generar_graficas(resultados)

    return resultados

# Ejecutar análisis
resultados_finales = ejecutar_analisis_completo()

🚀 Iniciando análisis completo...

📊 RESULTADOS DEL ANÁLISIS
🔋 Celda analizada: Análisis de Celda Solar
📅 Fecha de análisis: 2025-07-08 18:08:31
📊 Puntos de medición: 151

📈 PARÁMETROS PRINCIPALES:
  • Jsc (Densidad de corriente cortocircuito): 17.3342 mA/cm²
  • Voc (Voltaje circuito abierto): 0.7138 V
  • Imp (Corriente máxima potencia): 14.9119 mA/cm²
  • Vmp (Voltaje máxima potencia): 0.5800 V
  • Pmax (Potencia máxima): 8.6489 mW/cm²
  • FF (Factor de llenado): 69.91%
  • η (Eficiencia/PCE relativa): 86.49%


/tmp/ipykernel_33492/575434450.py:135: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 📖 PASO 4: Interpretación de resultados

### 🔍 ¿Cómo interpretar los resultados?

Una vez que hayas ejecutado el análisis, aquí te explicamos qué significan los números:

#### 📊 Parámetros principales:

**🔌 Isc (Corriente de cortocircuito)**
- Es la máxima corriente que puede generar la celda
- Se mide cuando el voltaje es cero (V = 0)
- Valores típicos: 0.1 - 10 A (depende del tamaño de la celda)

**⚡ Voc (Voltaje de circuito abierto)**
- Es el máximo voltaje que puede generar la celda
- Se mide cuando no hay corriente (I = 0)
- Valores típicos: 0.5 - 0.7 V (celda de silicio)

**🔋 Pmax (Potencia máxima)**
- Es la máxima potencia que puede generar la celda
- Se calcula como P = V × I en el punto óptimo
- Valores típicos: 0.01 - 5 W (depende del tamaño)

**📊 FF (Factor de llenado)**
- Indica qué tan "cuadrada" es la curva I-V
- Se calcula como FF = Pmax / (Isc × Voc)
- Valores típicos: 70-85% (celdas comerciales)

**🌟 η (Eficiencia)**
- Porcentaje de luz solar convertida en electricidad
- Se calcula como η = Pmax / (Irradiancia × Área)
- Valores típicos: 15-25% (celdas comerciales)

#### 🎯 ¿Qué valores son buenos?

| Parámetro | Excelente | Bueno | Regular | Malo |
|-----------|-----------|-------|---------|------|
| FF | >80% | 70-80% | 60-70% | <60% |
| Eficiencia | >20% | 15-20% | 10-15% | <10% |

#### 🔧 Posibles problemas:

- **FF bajo**: Puede indicar resistencia interna alta o defectos en la celda
- **Eficiencia baja**: Puede ser debido a material defectuoso o condiciones de medición
- **Isc muy bajo**: Posible problema con la iluminación o área activa
- **Voc muy bajo**: Posible problema con la calidad del material

---

**💡 Tip**: Compara tus resultados con datos de referencia para evaluar el rendimiento de tu celda solar.

## 🆘 Ayuda y soporte

### 📞 ¿Necesitas ayuda?

Si tienes preguntas o problemas con el notebook, aquí tienes opciones:

#### 🐛 Problemas comunes:

**"No se han cargado datos"**
- Asegúrate de haber ejecutado una de las opciones del PASO 1
- Descomenta la línea correspondiente en la celda elegida

**"Error al cargar archivo CSV"**
- Verifica que el archivo tenga extensión .csv
- Revisa que tenga al menos 2 columnas (voltaje y corriente)
- Asegúrate de que los datos sean números válidos

**"Las gráficas no se ven"**
- En Google Colab: deberían aparecer automáticamente
- En VS Code: asegúrate de tener las extensiones Python y Jupyter instaladas
- En Jupyter: ejecuta `%matplotlib inline` en una celda

**"Error al instalar paquetes"**
- En Google Colab: se instalan automáticamente
- En otros entornos: ejecuta `pip install numpy matplotlib scipy pandas`

#### 📚 Recursos adicionales:

- **📖 Manual completo**: Consulta el archivo `README.md` en el proyecto
- **🔧 Para desarrolladores**: Consulta `TECHNICAL_DOCS.md`
- **📊 Ejemplos**: Revisa los archivos `datos_ejemplo.csv` y `ejemplo2.csv`

#### 👥 Contacto:

- **📧 Email**: adrianarazo@outlook.com
- **🐙 GitHub**: Reporta problemas en el repositorio del proyecto
- **💬 Soporte académico**: Contacta directamente para preguntas específicas

#### 🔧 Opciones avanzadas:

Si eres programador y quieres usar las herramientas de línea de comandos:

- **Windows**: Ejecuta `install.bat` y luego `run.bat`
- **Linux/Mac**: Ejecuta `install.sh` y luego `run.sh`
- **Python directo**: Ejecuta `python graph_I_V.py`

---

### 🎓 Información del proyecto

**Autora**: Adriana Razo De León  
**Proyecto**: Análisis de Celdas Solares  
**Versión**: 2.0 (Notebook Interactivo)  
**Fecha**: 2025  
**Licencia**: Ver archivo LICENSE

---

## 🌟 ¡Gracias por usar el Analizador de Celdas Solares!

¡Esperamos que esta herramienta te sea útil para tus investigaciones y proyectos! 

🔋 ¡Que tengas excelentes resultados con tus celdas solares! 🔋